# Combat simulator for Beat Arena

Welcome to the Beat Arena Combat Simulator!!

To start, run the cell bellow to get the required functions. If asked, give permission to Colab to access your Drive files.

In [1]:
#@title
#importing the required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
#mounting google drive to colab
drive.mount('/content/gdrive')
file_name = '/content/gdrive/MyDrive/combat_simulator/BA - Cards Data - Cards Data.csv'
cards = pd.read_csv(file_name)
#removing the extra rows
cards= cards[cards['ID'].notna()]
#converting the percentage columns to floats
cards.iloc[:,18:30] = cards.iloc[:,18:30].replace('%', '', regex=True).astype(float)/100
#defining the function to create the builds
def get_build(cards_ids,levels = [1,1,1,1,1,1]):
    '''
    Parameters:
        cards: a list of IDs to use for the cards dataframe containing the cards for the build
        levels: a list of the levels for the cards in the build
    Return: a list containing the cards and the levels
    Description: used to create the cards that conform the build for a player and their corresponding levels  
    '''
    #getting the slice of the cards table
    build_cards = cards[cards['ID'].isin(cards_ids)]
    #adjusting the levels array to match the number of the cards
    levels_array = np.array(levels)[0:build_cards.shape[0]]
    return [build_cards,levels_array]
def calc_HP(level = 1, HP_boost = 0):
    '''
    Parameters:
        level: the player level
        HP_boost: the boost to the HP from the cards
    Return: the total HP
    Description: calculates the HP of a player using 200 as the HP in first level and then adding 30 per level, 
    finally adding the boost that is given by the cards in the build.
    '''
    total_HP = 200 + (level-1)*30 + HP_boost
    return total_HP
def calc_HP_boost(build):
    '''
    Parameters:
        build: the cards of the player and their levels
    Return: the total boost as an int
    Description: calculates the boost given to the HP given by the cards in the build
    '''
    boost = build[0]['healthPointBoost'].sum()
    return boost
def calc_dmg_base(build):
    '''
    Parameters:
        build: the cards of the player and their levels
    Return: a dataframe with the damage of all elements for all cards
    Description: calculates the base damage for the individual cards based on their level
    '''
    upgrade_functions = build[0]['upgradeIntFunction']
    upgrade_int = build[0]['upgradeIntFunctionValue']
    card_levels = build[1]
    # applying elemental_dmg to every column
    base_dmg = build[0][[
        'physicDamage','magicDamage','lightingDamage','fireDamage','poisonDamage','bleedDamage']].apply(
        elemental_dmg,args=(upgrade_functions, upgrade_int, card_levels))
    return base_dmg
def elemental_dmg(x, upgrade_functions, upgrade_int, card_levels):
    '''
    Parameters:
        x: is an elemental damage column of the cards in a build
        upgrade_functions: the column with the upgrade functions of the cards (it can be linear or exponential)
        upgrade_int: the column with the parameters for the upgrade function
        card_levels: the levels of the cards obtained from the build
    Return: the damage for that element for all cards
    Description: calculates the elemental damage based on the base damages of the cards and their level. This function
    is for use with "apply" in calc_dmg_base
    '''
    dmg = np.where(upgrade_functions == 'Linear',
                   x + (card_levels-1)*upgrade_int*x,
                   x *(upgrade_int**(card_levels-1)))
    return dmg
def calc_dmg_bonus(build):
    '''
    Parameters:
        build: the cards of the player and their levels
    Return: the damage bonuses of all elements
    Description: calculates the damage bonuses based on the individual cards in the build to use them in calc_dmg_total
    '''
    bonus_dmg = build[0][['physicDamageBonus','magicDamageBonus','fireDamageBonus',
                             'lightingDamageBonus','poisonDamageBonus','bleedDamageBonus']].sum().values
    return bonus_dmg
def calc_dmg_total(build):
    '''
    Parameters:
        build: the cards of the player and their levels
    Return: a dataframe with the total damage of all elements
    Description: calculates the total damage based on the cards in the build by adding the base damage and the bonus
    '''
    total_dmg = calc_dmg_base(build)+calc_dmg_base(build)*calc_dmg_bonus(build)
    return total_dmg
def calc_resistance(build):
    '''
    Parameters:
        build: the cards of the player and their levels
    Return: a dataframe with the resistances for all elements
    Description: calculates the total resistance based on the cards in the build by adding them
    '''
    resistances = build[0][[
        'physicResistancePorcentage','magicResistancePorcentage','fireResistancePorcentage',
                             'lightingResistancePorcentage','poisonResistancePorcentage','bleedResistancePorcentage']].sum().values
    return resistances
def get_criticals(build):
    '''
    Parameters:
        build: the cards of the player and their levels
    Return: a dataframe with the critical chances and critical multipliers for every card
    Description: returns the critical chance and multipliers for every card in the build
    '''
    criticals = build[0][['criticalChance','criticalMultiplier']]
    return criticals
def calc_dodge(build):
    '''
    Parameters:
        build: the cards of the player and their levels
    Return: the sum of the dodge chances for the weapons in the build
    Description: sums the dodge chances of all the cards in the build
    '''
    dodge = build[0][['dodgeChance']].sum()[0]
    return dodge
def calc_block(build):
    '''
    Parameters:
        build: the cards of the player and their levels
    Return: the sum of the block proportion for the weapons in the build
    Description: sums the block proportion of all the cards in the build
    '''
    block = build[0][['blockAmount']].sum()[0]
    return block
class player:
    '''
    Parameters: 
      name: string to name the player
      accuracy: an int or float representing the mean accuracy
      cards_idx: a list of cards indicies corresponding to Card_ID in the cards 
                 table
      level = an int representing player level, default 1
      card_levels = a list of ints representing the levels of each card in the 
                    build. Follows the same order as cards_idx. 
                    Default [1,1,1,1,1,1]
    Description: this class is to represent the objects that are used in the 
                 simulation to represent the players.
    '''
    def __init__(self, name, accuracy, cards_idx, level = 1, 
                 card_levels = [1,1,1,1,1,1]):
        #defining the attributes of the player
        self.name = name
        self.accuracy = accuracy
        self.level = level
        self.build = get_build(cards_idx, card_levels)
        self.cards = self.build[0]
        self.cards_levels = self.build[1]
        self.damage = calc_dmg_total(self.build)
        self.HP = calc_HP(level,calc_HP_boost(self.build))
        self.critical = get_criticals(self.build)
        self.block = calc_block(self.build)
        self.dodge = calc_dodge(self.build)
        self.resistance = calc_resistance(self.build)
        self.card_order = np.array(range(len(self.cards_levels))) #the order determines the sequence in which the
                                                                    #cards are played, it is initiated with a range
    def shuffle_cards(self):
        '''
        Return: none
        Description: shuffles the card order
        '''
        np.random.shuffle(self.card_order)
    def get_next_card_idx(self, n_round):
        '''
        Parameters:
            n_round: an int that is the current round number
        Return: the index of the card to be played in a round 
        Description: defines the index of the card to be placed based on the current round number and the order 
        of the cards
        '''
        next_card_idx = self.card_order[n_round%len(self.cards_levels)]
        return next_card_idx
        
def get_HP(player):
    '''
    Parameters:
        player: the player from which the HP is obtained
    Return: the HP of the player
    Description: extracts the max HP of the player
    '''
    HP = player.HP
    return HP
def get_player_card_idx(player,n_round):
    '''
    Parameters:
        player: the player from which the card index is obtained
    Return: the index of the card being played by the player
    Description: gets the index of the card being played by the player in the current round to use with 
    other functions
    '''
    card_idx = player.get_next_card_idx(n_round)
    return card_idx
def who_geos_first(player_A,player_B):
    '''
    Parameters:
        player_A and player_B: the two players in the encounter
    Return: the two players in a random order
    Description: determines the order of the players in a round
    '''
    random_order = np.random.randint(2)
    p1, p2 = [player_A,player_B][random_order], [player_A,player_B][1-random_order]
    return p1, p2
def is_ko(HP):
    '''
    Parameters:
        HP: the remaining health of a player
    Return: a boolean
    Description: determines if there is a knock out
    '''
    ko = (HP <= 0)
    return ko
def get_accuracy(player):
    '''
    Parameters:
        player: the player from which the accuracy is obtained
    Return: the simulated accuracy for the player for the current round
    Description: uses the the accuracy of the player to determine the accuracy of the player in that round. 
    The accuracy is sampled from a distribution centered around the value of the player.
    '''
    acc = player.accuracy
    acc = np.random.randint(acc-5,acc+5)
    if acc<0:
      acc = 0
    elif acc > 100:
      acc = 100
    return acc
def judge(accuracy1, accuracy2):
    '''
    Parameters:
        accuracy1: the accuracy of the attacker
        accuracy2: the accuracy of the deffender
    Return: an int that defines the outcome
        Outcome is 0 if the attacker missed (attacker accuracy < 60)
        Outcome is 1 if the attack was blocked (attacker accuracy <= defender accuracy)
        Outcome is 2 if the attack was a hit (attacker accuracy > defender accuracy)
    Description: compares the accuracies to define the outcome.
    '''
    if accuracy1<60:
        outcome = 0
    elif accuracy1<=accuracy2:
        outcome = 1
    else:
        outcome = 2
    return outcome
def is_crit(player, card_idx):
    '''
    Parameters:
        player: the attacker
        card_idx: the index of the card that the attacker is using
    Return: a boolean
    Description: defines if the attack was a critical hit based on the critical chance of the card used and returns
    a boolean indicating if it was a critical hit.
    '''
    crit_prob = player.critical.criticalChance.iloc[card_idx]/100
    return np.random.choice([True,False], p = [crit_prob,1-crit_prob])
def get_dmg(player, card_idx):
    '''
    Parameters:
        player: the attacker
        card_idx: the index of the card that the attacker is using
    Return: an array with the elemental damages for the card being used in the attack
    Description: extracts the damage of the card being used by the attacker
    '''
    dmg = player.damage.iloc[card_idx]
    return dmg
def apply_crit(player, card_idx):
    '''
    Parameters:
        player: the attacker
        card_idx: the index of the card that the attacker is using
    Return: an array with the elemental damages for the card being used in the attack
    Description: calculates the damage done by a critical attack of the card being used and returns the array of the
    elemental damages.
    '''
    critical_dmg = player.critical.criticalMultiplier.iloc[card_idx]*get_dmg(player,card_idx)
    return critical_dmg
def get_resistance(player):
    '''
    Parameters:
        player: the defender
    Return: an array with the elemental resistances of the defender
    Description: gets the resistances of the defender
    '''
    resistance = player.resistance
    return resistance
def get_block(player):
    '''
    Parameters:
        player: the defender
    Return: a float between 0 and 1 indicating the proportion of the damage blocked by the defender
    Description: gets the block proportion of the defender
    '''
    block = player.block
    return block
def attack(player1, player2, card_idx):
    '''
    Parameters:
        player1: the attacker
        player2: the deffender
        card_idx: the index of the card that the attacker is using
    Return: the damage dealt by the attacker
    Description: this function is the main function in every round of the encounter. It defines the outcome of the
    attack based on the accuracies (miss, block or hit), it then calculates the damage dealt based on the otucome, 
    the damage of the card used by the attacker, the block proportion of the defender and the resistance of 
    the defender.
    '''
    #getting the accuracies
    acc1 = get_accuracy(player1)
    acc2 = get_accuracy(player2)
    #getting the card damage
    card_dmg = get_dmg(player1, card_idx)
    #getting the accuracy modifier
    accuracy_modifier = acc1/100
    #calculating the total damage
    total_dmg = accuracy_modifier*card_dmg
    #calculating the damage if the attack is a critical hit
    if is_crit(player1, card_idx):
        total_dmg = accuracy_modifier*apply_crit(player1, card_idx)
    #getting the resistances of the defender
    resistance = get_resistance(player2) 
    #getting the block proportion of the defender
    block = get_block(player2) 
    #defining the outcome of the attack
    outcome = judge(acc1, acc2)
    #calculating the damage based on the outcome
    if outcome == 0:
        final_dmg = 0
    elif outcome == 1:
        final_dmg = total_dmg-resistance*total_dmg
        final_dmg = final_dmg * (1-block)
    else:
        final_dmg = total_dmg-resistance*total_dmg
    
    final_dmg = np.sum(final_dmg)
    return final_dmg
def encounter(player1,player2, max_rounds = 24):
    '''
    Parameters:
        player1 and player2: two player objects that represent the two simulated players that participate in 
        the encounter. 
    Return: a tuple with 6 elements:
                The winner: the name of the winning player as defined in the player object
                The first player: the name of the first player as defined in the player object
                The number of rounds played: an int
                The remaining health of winner: a float
                The remaining health of the loser: a float
                The remaining health of player1: an int
                The remaining health of player2: an int
                The win criteria: a string either ko (knock out defined as the losing player reached 0HP) or 
                                  time (the maximum number of rounds was reached) 
    Description: this function is the main function in the simulator. It simulates a whole encounter between the two
    players. It defines randomly the player who goes first, then simulates succesives attacks from each player every 
    round until a win criteria is met. These are win by knock out, a player reaches 0HP, and time, the maximum number
    of rounds is reached and the player with more health remaining is the winner. Ir returns a tuple with the outcomes
    of the simulated encounter in a tuple to be added as a row in a data frame with the results of the simulation 
    '''
    #defining which player goes first
    first, second = who_geos_first(player1, player2)
    #recording if player1 is first
    p1_first = (first.name == player1.name)
    #getting the starting HP
    first_HP = get_HP(first)
    second_HP = get_HP(second)
    #setting the rounf counter to 0
    round_n = 0
    #shuffleing the cards of both players
    first.shuffle_cards()
    second.shuffle_cards()
    #simulating the rounds
    while round_n < (max_rounds+1):
        #attack of the first player
        first_card = get_player_card_idx(first, round_n)
        first_attack = attack(first, second, first_card)
        #updating the HP of the second player
        second_HP -= first_attack
        #check if it was a knock-out
        if is_ko(second_HP):
            #defining the elements to return
            winner = first.name
            w_HP = first_HP
            l_HP = second_HP
            criteria = 'ko'
            if p1_first:
              p1_HP, p2_HP = first_HP, second_HP
            else:
              p1_HP, p2_HP = second_HP, first_HP
            return winner, first.name, round_n, w_HP, l_HP, p1_HP, p2_HP,criteria
        #attack of the second player
        second_card = get_player_card_idx(second, round_n)
        second_attack = attack(second, first, second_card)
        #updating the HP of the first player
        first_HP -= second_attack
        #check if it was a knock-out
        if is_ko(first_HP):
            #defining the elements to return
            winner = second.name
            w_HP = second_HP
            l_HP = first_HP
            criteria = 'ko'
            if p1_first:
              p1_HP, p2_HP = first_HP, second_HP
            else:
              p1_HP, p2_HP = second_HP, first_HP
            return winner, first.name, round_n, w_HP, l_HP, p1_HP, p2_HP, criteria
        round_n += 1
    #maximum number of rounds has been reached
    #if the first player wins
    if first_HP > second_HP:
      #defining the elements to return
      winner = first.name
      w_HP = first_HP
      l_HP = second_HP
      criteria = 'time'
      if p1_first:
        p1_HP, p2_HP = first_HP, second_HP
      else:
        p1_HP, p2_HP = second_HP, first_HP
      return winner, first.name, round_n, w_HP, l_HP, p1_HP, p2_HP, criteria
      #if the second player wins
    else:
      winner = second.name
      w_HP = second_HP
      l_HP = first_HP
      criteria = 'time'
      if p1_first:
        p1_HP, p2_HP = first_HP, second_HP
      else:
        p1_HP, p2_HP = second_HP, first_HP
      return winner, first.name, round_n, w_HP, l_HP, p1_HP, p2_HP, criteria
def run_sim():
    '''
    Parameters: none
    Return: a pandas DataFrame containing the results of the simulation where 
            each encounter is a row with the following columns:
              winner: the name of the winning player
              first: the name of the player that went first in the encounter
              n_rounds: the number of rounds that the encounter lasted
              winner_HP: the remaining HP of the winner
              loser_hP: the remaining HP of the loser
              pA_HP: the remaining HP of PlayerA
              pB_HP: the remaining HP of PlayerB
              criteria: a string either ko (knock out defined as the losing 
                        player reached 0HP) or time (the maximum number of 
                        rounds was reached) representing the win criteria 
    Description: this function is used as a guided interface to creating and 
                 executing a simulation. The players, the number of rounds per 
                 encounter and the number of encounters simulated are defined by
                 the user. The results of the simulation are stored in a 
                 DataFrame.
    '''
    #Defining the number of simulations
    print('How many encounters should be simulated?')
    while True:
      try: 
        n_encounters = int(input('Please enter the number:'))
        if n_encounters<0:
          print('Sorry, only non negative numbers are allowed')
          continue
        break
      except:
        print('Sorry, something went wrong, it seems that your input is not an integer')


    #Defining the total number of rounds in each encounter.
    print('What should be the maximum number of rounds of each encounter (when the "song" ends)?')
    while True:
      try: 
        rounds = int(input('Please enter the number:'))
        if rounds<0:
          print('Sorry, only non negative numbers are allowed')
          continue
        break
      except:
        print('Sorry, something went wrong, it seems that your input is not an integer')

    #Defining player A
    print('Define player_A')
    print('What is the accuracy of player_A?')
    #Defining the accuracy
    while True:
      try:
        A_acc = float(input('Please enter the mean accuracy (it should be a number between 0 and 100)'))
        if A_acc>100:
          print('Sorry, the accuracy has to be less than 100')
          continue
        if A_acc<0:
          print('Sorry, the accuracy has to be more than 0')
          continue
        break
      except:
        print('Sorry, something went wrong, it seems that your input is not a float')
    #Defining the level
    print('What level is player_A?')
    while True:
      try: 
        A_level = int(input('Please enter the number:'))
        if A_level<0:
          print('Sorry, only non negative numbers are allowed')
          continue
        break
      except:
        print('Sorry, something went wrong, it seems that your input is not an integer')
    #Defining the cards
    print('Which cards does player_A have')
    while True:
      A_cards = input('Please enter the IDs of the cards separated by commas (ex: CA22, CA23,CA24)')
      A_cards = A_cards.replace(' ','')
      A_cards = A_cards.split(sep = ',')
      if len(A_cards)<1:
        print('Sorry, a player needs to have at least one card')
        continue
      if len(A_cards)>6:
        print('Sorry, a player can not have more than 6 cards')
        continue
      check_id = False
      for i in A_cards:
        if i not in cards.ID.tolist():
          check_id = True
      if check_id:
        print('Sorry, something went wrong, it seems that one or more cards do not exist')
        continue
      break
    #Defining the levels of the cards
    print('What levels are these cards')
    while True:
      A_cards_levels = input('Please enter the levels of the cards separated by commas (ex: 1, 2,3)')
      A_cards_levels = A_cards_levels.replace(' ','')
      A_cards_levels = A_cards_levels.split(sep = ',')
      if len(A_cards_levels) != len(A_cards):
        print('Sorry, the number of levels must match the number of cards')
        continue
      try:
        A_cards_levels = [int(i) for i in A_cards_levels]
        if any( i<1 for i in A_cards_levels):
          print('Sorry, the level can not be less than 1')
          continue
        if any(i >5 for i in A_cards_levels):
          print('Sorry, the level can not be more than 5')
          continue
        break
      except:
        print('Sorry, something went wrong, it seems that some levels are not integers')
    try:
      player_A = player('player_A', A_acc, A_cards, A_level, A_cards_levels)
    except:
      print('Something went wrong with player A configuration')

    #Defining player B
    print('Define player_B')
    print('What is the accuracy of player_B?')
    #Defining the accuracy
    while True:
      try:
        B_acc = float(input('Please enter the mean accuracy (it should be a number between 0 and 100)'))
        if B_acc>100:
          print('Sorry, the accuracy has to be less than 100')
          continue
        if B_acc<0:
          print('Sorry, the accuracy has to be more than 0')
          continue
        break
      except:
        print('Sorry, something went wrong, it seems that your input is not a float')
    #Defining the level
    print('What level is player_B?')
    while True:
      try: 
        B_level = int(input('Please enter the number:'))
        if B_level<0:
          print('Sorry, only non negative numbers are allowed')
          continue
        break
      except:
        print('Sorry, something went wrong, it seems that your input is not an integer')
    #Defining the cards
    print('Which cards does player_B have')
    while True:
      B_cards = input('Please enter the IDs of the cards separated by commas (ex: CA22, CA23,CA24)')
      B_cards = B_cards.replace(' ','')
      B_cards = B_cards.split(sep = ',')
      if len(B_cards)<1:
        print('Sorry, a player needs to have at least one card')
        continue
      if len(B_cards)>6:
        print('Sorry, a player can not have more than 6 cards')
        continue
      check_id = False
      for i in B_cards:
        if i not in cards.ID.tolist():
          check_id = True
      if check_id:
        print('Sorry, something went wrong, it seems that one or more cards do not exist')
        continue
      break
    #Defining the levels of the cards
    print('What levels are these cards')
    while True:
      B_cards_levels = input('Please enter the levels of the cards separated by commas (ex: 1, 2,3)')
      B_cards_levels = B_cards_levels.replace(' ','')
      B_cards_levels = B_cards_levels.split(sep = ',')
      if len(B_cards_levels) != len(B_cards):
        print('Sorry, the number of levels must match the number of cards')
        continue
      try:
        B_cards_levels = [int(i) for i in B_cards_levels]
        if any( i<1 for i in B_cards_levels):
          print('Sorry, the level can not be less than 1')
          continue
        if any(i >5 for i in B_cards_levels):
          print('Sorry, the level can not be more than 5')
          continue
        break
      except:
        print('Sorry, something went wrong, it seems that some levels are not integers')
    try:
      player_B = player('player_B', B_acc, B_cards, B_level, B_cards_levels)
    except:
      print('Something went wrong with player B configuration')

    #Initializing a Data Frame for the outcomes of the simulations
    sim = pd.DataFrame({'winner': [],
                          'first': [],
                          'n_rounds': [],
                          'winner_HP': [],
                          'loser_HP': [],
                          'pA_HP': [],
                          'pB_HP': [],
                          'criteria': []})

    #Running the simulations
    for i in range(n_encounters):
        outcome = encounter(player_A, player_B, rounds)
        sim.loc[len(sim.index)] = outcome
    
    return sim

Mounted at /content/gdrive


# Simulation

Next, run the following cell to do the simulation. It will guide you through the process of defining the required parameters.

In [2]:
#@title
my_sim = run_sim()

How many encounters should be simulated?
Please enter the number:15
What should be the maximum number of rounds of each encounter (when the "song" ends)?
Please enter the number:12
Define player_A
What is the accuracy of player_A?
Please enter the mean accuracy (it should be a number between 0 and 100)90
What level is player_A?
Please enter the number:4
Which cards does player_A have
Please enter the IDs of the cards separated by commas (ex: CA22, CA23,CA24)CA22, CA24
What levels are these cards
Please enter the levels of the cards separated by commas (ex: 1, 2,3)1, 2, 3
Sorry, the number of levels must match the number of cards
Please enter the levels of the cards separated by commas (ex: 1, 2,3)1, 2
Define player_B
What is the accuracy of player_B?
Please enter the mean accuracy (it should be a number between 0 and 100)95
What level is player_B?
Please enter the number:4
Which cards does player_B have
Please enter the IDs of the cards separated by commas (ex: CA22, CA23,CA24)CA24
Wha

# Results
Finally, run this cell to see a summary of the results.

In [10]:
#@title
#Printing the number of wins per player
A_wins = len(my_sim[my_sim.winner == 'player_A'])
B_wins = len(my_sim[my_sim.winner == 'player_B'])
A_mean_HP = my_sim[my_sim.winner == 'player_A'].winner_HP.mean()
B_mean_HP = my_sim[my_sim.winner == 'player_B'].winner_HP.mean()
won_first = len(my_sim[my_sim.winner == my_sim.first])
n_encounters = len(my_sim)
song_end = len(my_sim[my_sim.criteria == 'time'])
print(f'Player A won: {A_wins} encounters, that is {A_wins/n_encounters:.2f} of the total')
print(f'Player B won: {B_wins} encounters, that is {B_wins/n_encounters:.2f} of the total')
print(f'The encounters lasted on average {my_sim.n_rounds.mean():.2f} rounds')
print(f'The song ended before any player being defeated in {song_end/n_encounters:.2f} of the encounters')
print(f'Player A had on average {A_mean_HP:.2f} HP when he won')
print(f'Player B had on average {B_mean_HP:.2f} HP when he won')

Player A won: 1 encounters, that is 0.07 of the total
Player B won: 14 encounters, that is 0.93 of the total
The encounters lasted on average 0.87 rounds
The song ended before any player being defeated in 0.00 of the encounters
Player A had on average 108.20 HP when he won
Player B had on average 100.74 HP when he won


Run this cell if you want to take a look at some example results

In [ ]:
#@title
my_sim